# Prepare environment

In [1]:
!pip install underthesea
!pip install jieba
!pip install gensim

     |████████████████████████████████| 7.5 MB 4.3 MB/s 
     |████████████████████████████████| 235 kB 42.7 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 3.1 MB 45.4 MB/s 
     |████████████████████████████████| 743 kB 47.6 MB/s 
     |████████████████████████████████| 61 kB 466 kB/s 
     |████████████████████████████████| 895 kB 33.5 MB/s 
     |████████████████████████████████| 596 kB 42.8 MB/s 
     |████████████████████████████████| 3.3 MB 57.7 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=ae5c4841c1c5ab4f733d2b37c0d3a5aa3530ca7b67a3676b6a2b47616ad9cb90
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install pyvi

     |████████████████████████████████| 8.5 MB 4.6 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd '/content/gdrive/MyDrive/LDS02_k271_NguyenMinhQuan/Problem_3_RecomendationSystem_TikiOnlineShopping'

/content/gdrive/MyDrive/LDS02_k271_NguyenMinhQuan/Problem_3_RecomendationSystem_TikiOnlineShopping


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re


# Bước 1: Hiểu vấn đề, ngữ cảnh

### Ngữ cảnh
Giả sử doanh nghiệp thương mại điện tử top 2 Việt Name, tên là Tiki, chưa xây dựng hệ thống gợi ý sản phẩm tiêu dùng. Hiện Tiki muốn xây dựng hệ thống gợi ý này, để phát triển doanh nghiệp và cạnh tranh với các doanh nghiêp khác.

Và chúng ta là người được mời để phát triển hệ thống này.

### Xác định vấn đề
#### Mục tiêu:
Xây dựng hệ thống gợi ý sản phẩm tiêu dùng cho doanh nghiệp thương mại điện tử.
#### Giải pháp:
+ Content base recomendation (Gợi ý dựa trên sự tương đồng giữa các sản phẩm)
+ Collaborative filtering recomendation (Gợi ý dựa trên sự tương đồng và hành vi của các khách hàng với nhau)

Trong notebook này tập trung giải quyết bài toán đầu teine là content base recommendation

# Bước 2: Thu thập, hiểu dữ liệu

Dữ liệu đã được cấp sẵng với các thông tin như sau:

Dữ liệu được cung cấp sẵn gồm có các tập tin:
ProductRaw.csv, ReviewRaw.csv chứa thông tin sản phẩm,
review và rating cho các sản phẩm thuộc các nhóm hàng
hóa như Mobile_Tablet, TV_Audio, Laptop, Camera,
Accessory.

Gồm các cột như sau: 
+ item_id: mã sản phẩm
+ name: tên sản phẩm 
+ description: mô tả sản phẩm 
+ price: giá sản phẩm 
+ list_price: giá sản phẩm chưa khuyến mãi (nguyên bán)
+ rating: số sao đánh giá sản phẩm (1: tệ, đến 5: tốt)
+ brand: thương hiệu của sản phẩm
+ group: loại sản phẩm
+ url: đường link của sản phẩm
+ img: link hình ảnh của sản phẩm

In [6]:
data = pd.read_csv('ProductRaw.csv')
data.head()

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.8,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,4.7,149000,350000,Acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...,https://salt.tikicdn.com/cache/280x280/ts/prod...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.8,5090000,8500000,Apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-apple-airpods-pro-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [7]:
#Select useful columns for content base recomendation algothirm
data = data[['item_id', 'name', 'description', 'brand', 'group']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_id      4404 non-null   int64 
 1   name         4404 non-null   object
 2   description  4401 non-null   object
 3   brand        4404 non-null   object
 4   group        4404 non-null   object
dtypes: int64(1), object(4)
memory usage: 172.2+ KB


In [8]:
#Check duplicate data
len(data['item_id'].unique()) == data.shape[0]

False

In [9]:
#Drop duplicate data
data = data.drop_duplicates(subset='item_id', keep = 'last')
len(data['item_id'].unique()) == data.shape[0]

True

In [10]:
pd.options.display.max_rows = 250

In [11]:
data['super_group'] = data['group'].apply(lambda x: x.split('-')[0])
data['super_group'].value_counts()

Thiết Bị Số                                             1065
Điện Tử                                                 1000
Máy Ảnh                                                  994
Laptop                                                   907
Điện Thoại                                               398
Hàng Quốc Tế/Laptop & Máy Vi Tính/Màn hình máy tính        2
Ô Tô                                                       2
Hàng Quốc Tế/Laptop & Máy Vi Tính/Linh kiện máy tính       2
Nhà Cửa                                                    1
Đồng hồ và Trang sức/Trang sức/Vòng tay                    1
Hàng Quốc Tế/Máy Ảnh                                       1
Name: super_group, dtype: int64

Đa số các sản phẩm đều là hàng tiêu dùng thiết bị số, công nghệ, ... Có 3 loại sản phẩm khác biệt hoàn toàn với các sản phẩm còn lại như: nhà cửa, đồng hồ và trang sức, ô tô
Loại sản phẩm chiếm số lượng nhiều nhất là máy ảnh, đồ dùng điện tử, thiết bị số, laptop.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4373 entries, 0 to 4403
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_id      4373 non-null   int64 
 1   name         4373 non-null   object
 2   description  4370 non-null   object
 3   brand        4373 non-null   object
 4   group        4373 non-null   object
 5   super_group  4373 non-null   object
dtypes: int64(1), object(5)
memory usage: 239.1+ KB


Dữ liệu gồm 4373 dòng tương ứng với 4373 sản phẩm:  
+ không có dữ liệu thiếu
+ các cột đều đúng format
+ riêng cột object là kiểu chuỗi, thì sẽ phải nối các cột này lại với nhau để tạo nên feature raw_text. 

# Bước 3: Tiền xử lý dữ liệu
Vì những feature chưa thông tin quan trọng đều là dữ liệu dạng , do đó, bước làm sạch dữ liệu sẽ tập trung vào làm sạch dữ liệu dạng text, bao gồm các bước như sau:  

+ Tổng hợp các dữ liệu text thành một feature
+ Loại bỏ các ký tự không hợp lệ, stopwords, ...
+ Chuyển thành vector chuẩn hóa 


In [13]:
#Create new feature
data['raw_text'] = data[[ 'name', 'description' ]].apply(lambda x: ' '.join([str(t) for t in x]), axis = 1)

data.head()

,item_id,name,description,brand,group,super_group,raw_text
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,Thiết Bị Số,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,Thiết Bị Số,Tai nghe bluetooth không dây F9 True wireless ...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,Thiết Bị Số,Chuột Không Dây Logitech M331 Silent - Hàng Ch...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,Acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,Thiết Bị Số,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,Apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,Thiết Bị Số,Tai Nghe Bluetooth Apple AirPods Pro True Wire...


Dựa vào bài toán pos tag của underthesea: https://github.com/undertheseanlp/underthesea/wiki/M%C3%B4-t%E1%BA%A3-d%E1%BB%AF-li%E1%BB%87u-b%C3%A0i-to%C3%A1n-POS-Tag
ta có thể tạo preprocessor như sau:

+ Bước đầu tiên: lower, loại bỏ url, ký tự đặc biệt ra khỏi document
+ Bước hai: Tokenize document và gán nhãn loại từ cho từng token
+ Bước ba: Từ list các token và các nhãn, loại bỏ các stopwords, và các nhãn không quan trọng
+ Bước bốn: trả về kết quả là danh sách các từ đã được làm sạch

In [14]:
#Create a preprocessor
class VietnamesePreprocessor():
  def __init__(self, additional_stopwords = None):
    from pyvi import ViTokenizer, ViPosTagger
    import re
    import numpy as np

    with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
      self.stop_words = list(f.read().split('\n'))

    if additional_stopwords:
      if (type(additional_stopwords) =='str'):
        self.stop_words.append(additional_stopwords)
      else:
        self.stop_words.extend(additional_stopwords)

    self.stop_words = np.array(self.stop_words)
    self.tag_remove = ( 'C', 'F', 'I', 'L', 'M', 'Ny', 'Np', 'X', 'Nu', 'E') 
    self.remove_char = ['--', '//', '[[', '((', ']]', '))']
    self.nlp = lambda x: ViPosTagger.postagging(ViTokenizer.tokenize(x))
  
  def clean_text(self, text):
    import re
    text = re.sub('http://\S+|https://\S+', ' ', text)
    text = text.lower()
    text = text.replace('\n', ' ') 

    return text.lower()

  def preprocess(self, document):

    document = self.clean_text(document)

    postag_ = self.nlp(document)

    bag_words = []

    for word, tag in zip(postag_[0], postag_[1]): 
      if word not in self.stop_words and tag not in self.tag_remove and len(word) > 1 and word not in self.remove_char:
        bag_words.append(word)

    return bag_words 



### Test class

In [15]:
preprocessor = VietnamesePreprocessor(additional_stopwords=['công_nghệ', 'điện_tử', 'thiết_bị_số', 'điện', 'xuất_xứ', 'chi_tiết', 'thông_tin', 'thương_hiệu', 'màu_sắc', 'nâng_cấp',
                                                            'trọng_lượng', 'sản_phẩm', 'tỉ_lệ', 'nâng_cấp', 'thiết_kế', 'tiki', 'hàng', 'giao', 'phương_thức', 'chi_phí', 'phí',
                                                            'vận_chuyển', 'giá', 'thiết_bị', 'chất', 'thuế', 'luật'])

print(preprocessor.preprocess("Hôm nay tôi đi\nmua đồ ăn ở chợ, gặp ông bán cá rô phi\t, ổng nói tiếng việt, mua 100 con cá trả một cụt vàng"))
print(preprocessor.preprocess(data.iloc[0]['raw_text']))

['hôm_nay', 'mua', 'đồ', 'ăn_ở', 'chợ', 'cá_rô_phi', 'tiếng', 'mua', 'cá', 'cụt', 'vàng']
['tai_nghe', 'bluetooth', 'inpods', 'cảm_biến', 'vân', 'chống', 'đa_dạng', 'lựa_chọn', 'dung_lượng', 'pin', 'pin', 'nhạc', 'liên_tục', 'sạc', 'chờ', 'bluetooth', 'oem', 'trung', 'quốc', 'độ', 'nhạy_cảm_biến', 'vân', 'model', 'i12', 'jack', 'cắm', 'usb', 'cable', 'nhạc', 'liên_tục', 'sku', 'mô_tả', 'inpod', 'phiên_bản', 'tai_nghe', 'bluetooth', 'chuẩn', 'tai', 'airpod', 'hãng', 'lược', 'nút', 'bấm', 'thân', 'tai', 'thay', 'nút', 'cảm_ứng', 'dễ_dàng', 'thuận_tiện', 'thao_tác', 'cuộc_gọi', 'nhạc', 'dễ_dàng', 'chạm', 'bluetooth', 'kết_nối', 'vô_cùng', 'ổn_định', 'bluetooth', 'kết_nối', 'vô_cùng', 'ổn_định', 'tai_nghe', 'kết_nối', 'dock', 'sạc', 'âm', 'thời_lượng', 'pin', 'cải_thiện', 'tối_ưu', 'dock', 'sạc', 'tiện_lợi', 'sạc', 'pin', 'tai_nghe', 'tương_thích', 'bluetooth', 'xiaomi', 'samsung', 'apple', 'huawei', 'oppo', 'vivo', 'lenovo', 'tablet', 'nhạc', 'liên_tục', 'sạc', 'chờ', 'bao_gồm', 'hiện_hàn

# Bước 4: Tạo model

In [16]:
#Create a similarity comparator

class VietnameseSimilarity():
  def __init__(self):
    pass

    
  def make_dictionary(self, corpus):
      self.corpus = list(corpus)
      print(type(self.corpus))
      self.dictionary = corpora.Dictionary(self.corpus)

  def to_vectors(self):
      from gensim.models import TfidfModel
      self.wordVectors = [self.dictionary.doc2bow(bag_words) for bag_words in self.corpus]
      self.tfidf = TfidfModel(self.wordVectors)
      return self.tfidf
  
  def to_similarity_matrix(self):
    from gensim import similarities
    self.similarityMatrix = similarities.SparseMatrixSimilarity(self.tfidf[self.wordVectors], num_features=len(self.dictionary.token2id))
    return self.similarityMatrix 

  def fit(self, corpus):
    self.make_dictionary(corpus)
    self.to_vectors()
    return self.to_similarity_matrix()

  def compare(self, new_data):
    bag_words = new_data
    new_vec = self.dictionary.doc2bow(bag_words)
    return self.similarityMatrix[self.tfidf[new_vec]]

  def save_model(self, dirPath):
    self.tfidf.save(dirPath + "/tfidf")
    self.dictionary.save(dirPath + "/dict")
    self.similarityMatrix.save(dirPath + '/index')

In [17]:
#Create a recommendator model
class ContentBaseRecommendator():
  def __init__(self):
    self.preprocessor = VietnamesePreprocessor(additional_stopwords=['công_nghệ', 'điện_tử', 'thiết_bị_số', 'điện', 'xuất_xứ', 'chi_tiết', 'thông_tin', 'thương_hiệu', 'màu_sắc', 'nâng_cấp',
                                                            'trọng_lượng', 'sản_phẩm', 'tỉ_lệ', 'nâng_cấp', 'thiết_kế', 'tiki', 'hàng', 'giao', 'phương_thức', 'chi_phí', 'phí',
                                                            'vận_chuyển', 'giá', 'thiết_bị', 'chất', 'thuế', 'luật'])
    self.comparator = VietnameseSimilarity()

  def fit(self, document, listItemsId):
    self.items = np.array(listItemsId)

    self.corpus = document.apply(self.preprocessor.preprocess)

    return self.comparator.fit(self.corpus)

  def recommend(self, newContent: str, top_n = 10):
    bag_words = self.preprocessor.preprocess(newContent)
    similarities = self.comparator.compare(bag_words)
    predData = pd.Series(data = similarities, index = self.items)
    predData = predData.where(predData < 1)
    return predData.nlargest(top_n)

  def save_model(self, dirPath):
    self.comparator.save_model(dirPath)
    np.save(dirPath + '/itemsId', self.items)
    

  

In [18]:
#Init model
model = ContentBaseRecommendator() 
#Fit to train data
model.fit(data['raw_text'], data['item_id'])


<class 'list'>


In [19]:
item_id = 48102821
item_description = data.loc[data['item_id'] == item_id, 'raw_text'][0]
item_description

'Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời gian pin - Thời gian nghe nhạc liên tục từ 2.5-4h - Thời gian sạc đầy chỉ khoảng 60p - Thời gian chờ lên tới 140 giờ\nBluetooth 5\nThương hiệu OEM\nXuất xứ thương hiệu Trung Quốc\nĐộ nhạy cảm biến vân tay\nModel i12\nLoại Jack cắm USB Cable\nTrọng lượng 300g\nThời gian sử dụng\n- Thời gian nghe nhạc liên tục từ 2.5-4h\nSKU 4096608751631\nMÔ TẢ SẢN PHẨM\nINPOD 12 là phiên bản nâng cấp mới nhất , tai nghe Bluetooth 5.0 có thiết kế tỉ lệ chuẩn 1:1 với tai Airpod chính hãng\nLược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn\nThao tác nhận cuộc gọi đến, chuyển nhạc...một cách dễ dàng chỉ bằng 1 chạm\nBluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- Bluetooth 5.0 mới nhất cho kết nối vô cùng ổn định\n- 2 tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc\n- Chất âm và thời lượng pin được cải thiện tối

In [20]:
recommend_list = model.recommend(item_description)
recommend_list

56365197    0.475552
48273751    0.433855
22413470    0.430999
50319688    0.430274
52333193    0.417261
35723184    0.410734
56885678    0.407965
26348659    0.405509
52889826    0.403976
35373097    0.399763
dtype: float32

In [21]:
data.loc[data['item_id'].isin(recommend_list.index), ['item_id','name']]

,item_id,name
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...
23,35373097,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
30,35723184,Tai nghe Inpod i12 TWS Bluetooth 5.0 cho iPhon...
66,52889826,"Tai Nghe Bluetooth Không Dây 5.0, AMOI F9, Cảm..."
119,26348659,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
134,56885678,Tai Nghe Bluetooth TWS F9 Tai Nghe Nhét Hai T...
147,22413470,Tai Nghe Bluetooth Air.podes Cảm Ứng Công Nghệ...
228,50319688,Tai Nghe Bluetooth Mini I12 Tws V5.0 (Trắng) N...
639,56365197,"Tai nghe bluetooth không dây i12 TWS 5.0, thiế..."
719,48273751,Tai nghe Blutooth 5.0 kiêm dock sạc dự phòng- ...


#### thử gợi ý sản phẩm bằng một content hoàn toàn mới so với data
link: https://tiki.vn/ban-phim-bosston-803-lot-p11242302.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.84974_Y.757948_Z.1491128_CN.phim-803%252Blot&itm_medium=CPC&itm_source=tiki-ads&spid=37793205

In [22]:
#predict new content
item_description = '''
Thông Số Kỹ Thuật: Màu sắc: Đen - Xám Chữ khắc Laser chống bay Kết nối: USB 2.0, tương thích mọi hệ điều hành Độ bền phím > 10.000.000 lượt bấm
Giới thiệu sản phẩm Bàn phím giả cơ Bosston 803 Led 7 màu Bàn phím led chuyên game giả cơ Bosston 803 với thiết kế tựa như 1 bàn phím cơ với đường nét và kiểu dáng rất phá cách, hệ thống đèn led 7 màu cực kỳ ấn tượng trãi đều trên mặt phím, giúp người dùng, đặc biệt là các game thủ có thể thoải mái sử dụng vào ban đêm. Điều đặc biệt ở bàn phím Bosston 803 là các nút bàn phím được thiết kế cao hơn so với các bàn phím thông thường khác, với khoảng cách từ mặt bàn phím lên đến mặt nút bấm là 7mm. Tạo một không gian trống bên dưới các nút phím giúp người dùng có cảm giác rất êm khi gõ, giống như đang sử dụng một bàn phím cơ thật sự. Ngoài ra với thiết kế phím cao như trên sẽ rất dễ dàng khi vệ sinh, hoặc khi vô tình đỗ nước vào cũng không bị ảnh hưởng các vi mạch bên trong. Giúp tránh những hư hỏng đáng tiếc có thể xảy ra với bàn phím. Thông Số Kỹ Thuật: - Màu sắc: Đen - Xám - Chữ khắc Laser chống bay - Kết nối: USB 2.0, tương thích mọi hệ điều hành - Độ bền phím > 10.000.000 lượt bấm - Kích thước : 445 * 130 * 35 mm - Đèn led nền và led phím với 7 màu bắt mắt. - Số Phím : 104 phím - Antighost bấm 1 lần được 19 phím - Phím giả cơ nghe âm thanh rất thanh và êm 
Giá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Bên cạnh đó, tuỳ vào loại sản phẩm, hình thức và địa chỉ giao hàng mà có thể phát sinh thêm chi phí khác như phí vận chuyển, phụ phí hàng cồng kềnh, thuế nhập khẩu (đối với đơn hàng giao từ nước ngoài có giá trị trên 1 triệu đồng).....
'''
recommend_list = model.recommend(item_description)
recommend_list

10590207    0.724216
4252075     0.689986
1044890     0.684991
3953455     0.634175
11241055    0.633454
25732951    0.615237
2883909     0.575232
588800      0.567281
16385902    0.527404
11582921    0.481812
dtype: float32

In [23]:
data.loc[data['item_id'].isin(recommend_list.index), ['item_id','name']]

,item_id,name
89,4252075,Bộ Bàn Phím Giả Cơ và Chuột Chuyên Game G21 Le...
146,16385902,Bàn phím giả cơ chuyên game cao cấp G700
259,3953455,Bộ Bàn Phím Giả Cơ và Chuột Chuyên Game G21 Le...
266,1044890,Bàn Phím Giả Cơ Chuyên Game Bosston 803 LED (7...
599,25732951,Bàn phím giả cơ chuyên Game R8 1822 Led 7 màu ...
600,11241055,Bàn Phím Giả Cơ G21 LED
2607,2883909,Bộ Bàn Phím Chuột Có Dây Bosston 837 Tích Hợp ...
2730,588800,Bộ Bàn Phím Giả Cơ Và Chuột Chuyên Game R8 191...
2859,10590207,Bàn Phím Giả Cơ Chuyên Game R8 1822 LED 7 Màu ...
2866,11582921,[TẶNG LÓT CHUỘT] Bộ bàn phím giả cơ và chuột c...


# Bước 5: Báo cáo kết quả

Hệ thống gợi ý sản phẩm bán lẻ trên tiki được xây dựng model như ở trên.

Hệ thống hoạt động dựa trên thuật toán: tfidf-cosine similarity, đưa ra gợi ý,sản phẩm tương đồng dựa trên tên và mô tả của sản phẩm.

In [24]:
#Save model
import dill as pickle
filename = '/content/gdrive/MyDrive/LDS02_k271_NguyenMinhQuan/Problem_3_RecomendationSystem_TikiOnlineShopping/model/gensim_contentbase.pkl'
pickle.dump(model, open(filename, 'wb'))

In [25]:
#Test model
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model.recommend(item_description)

10590207    0.724216
4252075     0.689986
1044890     0.684991
3953455     0.634175
11241055    0.633454
25732951    0.615237
2883909     0.575232
588800      0.567281
16385902    0.527404
11582921    0.481812
dtype: float32